# [PALSAR-2 の画像を取得する](https://www.tellusxdp.com/ja/howtouse/dev/20200221_000244.html)

In [1]:
import os
import json

fpth_secret = os.path.join(os.environ['HOME'], 'working/jupyter/secret.json')
with open(fpth_secret) as f:
    secret = json.load(f)
TOKEN = secret['tellus']
HEADERS = {
    'Authorization': 'Bearer ' + TOKEN,
    'content-type': 'application/json'
}

In [2]:
import requests

def search_scene_crossing_dataset(params):
    requirement_filter = [
        'is_order_required',
        'datasets',
        'intersects',
        'query',
        'sortby',
        'paginate'
    ]
    keys = list(filter_request.keys())
    if sorted(requirement_filter) == sorted(keys):
        url = 'https://www.tellusxdp.com/api/traveler/v1'
        url += '/data-search/'
        r = requests.post(url, headers=HEADERS, data=json.dumps(params))
        return r.json()
    else:
        excess_filters = [k for k in keys if k not in requirement_filter]
        if excess_filters:
            print(f'{excess_filters}: invalid filter')
        deficiency_filters = [f for f in requirement_filter if f not in keys]
        if deficiency_filters:
            print(f'{deficiency_filters}: please set filter')
        return None

In [3]:
filter_request = {
    'is_order_required': False,
    'datasets': [],
    'intersects': {
        "type": "Polygon",
        "coordinates": [
            [
                [139.967, 35.771],
                [140.573, 35.682],
                [140.71, 36.293],
                [140.099, 36.382],
                [139.967, 35.771],
            ]
        ],
    },
    'query': {
        "sar:frequency_band": {"eq": "L"},
        "processing:level": {"eq": "L2.1"},
        "gsd": {"eq": 3},
        "sar:polarizations": {"eq": "HH"},
        "sar:instrument_mode": {"eq": "UBS"},
    },
    'sortby': [
        {
            "field": "properties.end_datetime",
            "direction": "desc"
        }
    ],
    'paginate': {
        "size": 10,
        "cursor": None
    }
}

scenes = search_scene_crossing_dataset(filter_request)

In [4]:
print(json.dumps(scenes, indent=2))

{
  "type": "FeatureCollection",
  "features": [
    {
      "dataset_id": "b0e16dea-6544-4422-926f-ad3ec9a3fcbd",
      "geometry": {
        "coordinates": [
          [
            [
              140.5,
              36.31
            ],
            [
              141.038,
              36.446
            ],
            [
              140.805,
              37.044
            ],
            [
              140.263,
              36.907
            ],
            [
              140.5,
              36.31
            ]
          ]
        ],
        "type": "Polygon"
      },
      "id": "37461e12-df50-4899-8755-f9a29b267e7f",
      "type": "Feature",
      "properties": {
        "sar:frequency_band": "L",
        "processing:level": "L2.1",
        "sat:relative_orbit": 117,
        "palsar2:beam": "U3-12",
        "start_datetime": "2022-06-28T13:42:54.101000+00:00",
        "end_datetime": "2022-06-28T13:43:04.101000+00:00",
        "tellus:name": "ALOS2437280770-220628",
    

In [5]:
def fetch_files_list(scenes, idx):
    try:
        url_params = {
            'dataset_id': scenes['features'][idx]['dataset_id'],
            'data_id': scenes['features'][idx]['id']
        }
    except IndexError as e:
        print(e)
        return None
    url = 'https://www.tellusxdp.com/api/traveler/v1'
    url += '/datasets/{dataset_id}/data/{data_id}/files'.format(**url_params)
    
    r = requests.get(url, headers=HEADERS)
    return r.json()

def fetch_file_info(scenes, idx, file_id):
    try:
        url_params = {
            'dataset_id': scenes['features'][idx]['dataset_id'],
            'data_id': scenes['features'][idx]['id'],
            'file_id': file_id
        }
    except IndexError as e:
        print(e)
        return None
    url = 'https://www.tellusxdp.com/api/traveler/v1'
    url += '/datasets/{dataset_id}/data/{data_id}/files/{file_id}/'.format(**url_params)
    
    r = requests.get(url, headers=HEADERS)
    return r.json()

def fetch_download_url(scenes, idx, file_id):
    try:
        url_params = {
            'dataset_id': scenes['features'][idx]['dataset_id'],
            'data_id': scenes['features'][idx]['id'],
            'file_id': file_id
        }
    except IndexError as e:
        print(e)
        return None
    url = 'https://www.tellusxdp.com/api/traveler/v1'
    url += '/datasets/{dataset_id}/data/{data_id}/files/{file_id}/download-url/'.format(**url_params)
    
    r = requests.post(url, headers=HEADERS)
    return r.json()['download_url']

idx = 2
file_id = 4
file = fetch_files_list(scenes, idx)
print(json.dumps(file, indent=2))
file_info = fetch_file_info(scenes, idx, file_id)
print(json.dumps(file_info, indent=2))
download_url = fetch_download_url(scenes, idx, file_id) 
print(download_url)

SyntaxError: invalid syntax (1570369781.py, line 53)

In [ ]:
from tqdm import tqdm
file_name = file_info['name']
file_size = file_info['size_bytes']
print('ファイル名: ', file_name)
print('ファイルサイズ: ', file_size)

if not os.path.exists(file_name):
    response = requests.get(download_url, stream=True)
    progress_bar = tqdm(total=file_size, unit='B', unit_scale=True)
    with open(file_name, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
                f.flush()
                progress_bar.update(len(chunk))
        progress_bar.close()
            
from skimage import io
%matplotlib inline
# io.imshow(file_name)

In [ ]:
from tifffile import TiffFile
import numpy as np

with TiffFile(file_name) as tif:
    mmap = tif.asarray(out='memmap')
print(mmap.shape)
img = np.asarray(mmap[16000:20000, 18000:24000])
io.imshow(img)